<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


  Preparing metadata (pyproject.toml) ... -

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6231de61fd25658036156fffb6958e23532526f53cd065f97417b79b9b27d416
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


...

/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


/opt/hostedtoolcache/Python/3.14.2/x64/lib/python3.14/site-packages/yfinance/scrapers/history.py:173: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()


Run date time (IST): 2026-02-02 10:58:00
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.34 C
-------------------
Today PnL: -1.99 L (-1.46%)
Current PnL: -35.04 L (-22.35%)
CY Booked + Current PnL: -20.30 L (-12.95%)
-------------------
Total profit:  76.66 K
Total loss:  -35.81 L
-------------------
Total Booked + Current PnL: 6.68 L (5.14%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.96%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 1.01 C (74.78%)
Deployed:  1.30 C
Current:  1.34 C
CAGR/XIRR %: 1.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.65,2.26,16.59,19.23,25526.0,3404.0,153864.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,24.02,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-1.80,-26.22,35.66,0.09,26515.0,-26421.0,74356.0,72.68,34.0,M-SC,6.48,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
50,MASFIN,398.61,-5.76,-7.39,31.79,22.05,28845.0,-7245.0,90735.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.68,31.89,XR,ATH,FINANCE
33,HCLTECH,1908.19,-2.86,8.53,14.11,23.83,37046.0,20627.0,262551.0,13.18,50.0,X-LC,7.29,13.0,0.56,1.95,25.97,X40,ATH,IT
51,MEDANTA,1486.00,6.01,-8.47,35.56,24.08,41320.0,-10753.0,116197.0,-15.92,45.0,X-SC,9.49,91.0,-0.26,0.86,6.43,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,1486.0,6.01,-8.47,35.56,24.08,41320.0,-10753.0,116197.0,-15.92,45.0,X-SC,9.49,91.0,-0.26,0.86,6.43,XY24,NTT,HEALTHCARE
15,CAMPUS,393.0,4.85,-11.98,51.42,33.28,75939.0,-20091.0,147684.0,-29.13,54.0,M-SC,5.77,221.0,-0.26,1.10,14.47,XY24,NTT,FOOTWEAR
68,STARHEALTH,761.0,4.77,-8.66,61.23,47.26,157219.0,-24355.0,256768.0,24.55,64.0,H-SC,3.32,174.0,-0.15,1.91,37.57,XY24,NTT,INSURANCE
65,SIEMENS,4671.5,3.87,-16.81,50.87,25.51,78754.0,-31280.0,154815.0,-4.98,57.0,H-LC,4.13,50.0,-0.40,1.15,14.09,AR,ATH,ELECTRICAL
2,ABBOTINDIA,35195.0,3.04,-7.93,28.24,18.07,46503.0,-14185.0,164670.0,-21.26,44.0,X-MC,8.75,64.0,-0.31,1.23,7.12,X40,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.00,-9.25,-5.57,31.53,24.20,90881.0,-17012.0,288238.0,34.71,38.0,X-SC,2.51,99.0,-0.19,2.14,20.72,X40N,NTT,FINANCE
46,JIOFIN,387.00,-6.31,-23.30,62.30,24.48,127757.0,-62298.0,205067.0,-24.57,17.0,X-LC,7.68,37.0,-0.49,1.53,18.86,XY24,BTT,FINANCE
29,GREENPANEL,537.00,-6.06,-42.64,149.54,43.14,157044.0,-78060.0,105018.0,135.14,43.0,M-SC,17.39,231.0,-0.50,0.78,2.70,XY24,NTT,MISC
50,MASFIN,398.61,-5.76,-7.39,31.79,22.05,28845.0,-7245.0,90735.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.68,31.89,XR,ATH,FINANCE
18,COALINDIA,484.83,-5.65,2.26,16.59,19.23,25526.0,3404.0,153864.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,24.02,XY25,ATH,MINING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-0.58,-10.49,132.28,107.91,174812.0,-15491.0,132153.0,-27.04,36.0,M-SC,10.86,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
72,TATAELXSI,9161.00,1.05,-22.47,70.29,32.03,71843.0,-29625.0,102210.0,-16.04,48.0,H-SC,6.52,157.0,-0.41,0.76,14.48,OX40N,NTT,IT
66,SIS,528.00,2.40,-23.21,57.92,21.26,49382.0,-25773.0,85259.0,2012.44,54.0,H-SC,4.82,163.0,-0.52,0.63,15.19,OX40N,NTT,MISC
48,KANSAINER,340.00,-1.03,-25.54,52.40,13.47,105214.0,-68877.0,200790.0,-69.48,38.0,H-SC,5.00,159.0,-0.65,1.49,2.10,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.80,-26.22,35.66,0.09,26515.0,-26421.0,74356.0,72.68,34.0,M-SC,6.48,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.65,2.26,16.59,19.23,25526.0,3404.0,153864.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,24.02,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-1.52,-1.09,47.17,45.57,122061.0,-2851.0,258768.0,-36.40,64.0,H-MC,6.90,121.0,-0.02,1.93,14.91,AR,NTT,ELECTRICAL
84,WIPRO,420.00,1.56,-1.32,74.56,72.27,124677.0,-2231.0,167217.0,-15.15,44.0,M-LC,2.56,101.0,-0.02,1.24,7.48,XR,NTT,IT
39,INDIAMART,4810.62,0.66,-5.91,119.70,106.72,138911.0,-7287.0,116049.0,-53.49,48.0,H-SC,9.07,138.0,-0.05,0.86,16.09,AR,ATH,MISC
26,FINCABLES,1641.55,-0.58,-10.49,132.28,107.91,174812.0,-15491.0,132153.0,-27.04,36.0,M-SC,10.86,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-2.60,-8.52,48.26,35.63,92523.0,-17856.0,191718.0,-21.61,36.0,H-MC,5.51,119.0,-0.19,1.43,6.73,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,387.00,-6.31,-23.30,62.30,24.48,127757.0,-62298.0,205067.0,-24.57,17.0,X-LC,7.68,37.0,-0.49,1.53,18.86,XY24,BTT,FINANCE
13,BERGEPAINT,680.00,-1.22,-18.54,48.72,21.14,90223.0,-42153.0,185186.0,-24.35,18.0,X-MC,7.35,75.0,-0.47,1.38,0.00,XY24,NTT,PAINTS
12,BATAINDIA,2096.00,-0.40,-45.01,145.87,35.20,103215.0,-57912.0,70758.0,-7.29,20.0,X-SC,19.89,93.0,-0.56,0.53,0.55,X40,NTT,FOOTWEAR
6,ASIANPAINT,3460.25,-1.17,-16.11,44.18,20.95,93308.0,-40568.0,211200.0,-16.94,23.0,X-LC,10.58,36.0,-0.43,1.57,13.78,X40,ATH,PAINTS
44,ITC,452.00,1.11,-21.04,40.31,10.78,76487.0,-50572.0,189746.0,-53.00,24.0,X-LC,17.74,1.0,-0.66,1.41,1.11,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.58,0.98,-13.26,36.68,18.55,120339.0,-50159.0,328078.0,-25.24,50.0,X-LC,0.54,3.0,-0.42,2.44,11.62,X40,ATH,IT
81,VBL,671.64,-0.32,-5.96,44.27,35.67,131491.0,-18821.0,297021.0,-17.17,37.0,X-LC,1.82,5.0,-0.14,2.21,7.07,X40N,ATH,FMCG
34,HINDUNILVR,2922.00,-2.37,-9.87,26.12,13.67,60515.0,-25383.0,231680.0,-16.61,42.0,X-LC,2.25,9.0,-0.42,1.72,9.37,XY25,NTT,FMCG
5,ANGELONE,3033.00,-9.25,-5.57,31.53,24.20,90881.0,-17012.0,288238.0,34.71,38.0,X-SC,2.51,99.0,-0.19,2.14,20.72,X40N,NTT,FINANCE
36,ICICIGI,2252.93,-0.31,-3.59,23.79,19.34,45896.0,-7189.0,192920.0,-23.80,35.0,X-MC,2.74,71.0,-0.16,1.44,11.77,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-4.02,-32.30,142.46,64.15,229503.0,-76851.0,161100.0,-60.24,29.0,X-SC,7.28,84.0,-0.33,1.20,0.00,XY24,BTT,CEMENT
10,BAJAJHLDNG,14451.52,-2.11,-6.11,37.62,29.22,67158.0,-11608.0,178517.0,-8.67,35.0,X-LC,13.86,29.0,-0.17,1.33,0.00,X40,ATH,FINANCE
13,BERGEPAINT,680.00,-1.22,-18.54,48.72,21.14,90223.0,-42153.0,185186.0,-24.35,18.0,X-MC,7.35,75.0,-0.47,1.38,0.00,XY24,NTT,PAINTS
32,HAVELLS,2062.85,-2.14,-19.49,64.04,32.08,169114.0,-63916.0,264075.0,-23.56,25.0,X-MC,7.87,69.0,-0.38,1.96,0.00,X40,ATH,ELECTRICAL
53,PGHH,17907.65,0.06,-11.87,52.26,34.19,98341.0,-25344.0,188176.0,-38.01,28.0,X-MC,7.36,60.0,-0.26,1.40,0.06,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.55,-31.27,107.84,42.84,48179.0,-20330.0,44676.0,-54.82,47.0,X-SC,21.61,83.0,-0.42,0.33,5.01,XY24,NTT,MISC
12,BATAINDIA,2096.00,-0.40,-45.01,145.87,35.20,103215.0,-57912.0,70758.0,-7.29,20.0,X-SC,19.89,93.0,-0.56,0.53,0.55,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,-4.01,-51.37,213.18,52.31,150492.0,-74566.0,70594.0,-48.21,44.0,X-SC,11.46,92.0,-0.50,0.53,4.98,X40N,NTT,FOOTWEAR
51,MEDANTA,1486.00,6.01,-8.47,35.56,24.08,41320.0,-10753.0,116197.0,-15.92,45.0,X-SC,9.49,91.0,-0.26,0.86,6.43,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,0.70,-20.13,79.45,43.32,103349.0,-32792.0,130080.0,-31.39,45.0,X-SC,9.03,90.0,-0.32,0.97,5.62,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,1.11,-21.04,40.31,10.78,76487.0,-50572.0,189746.0,-53.00,24.0,X-LC,17.74,1.0,-0.66,1.41,1.11,X40,NTT,FMCG
75,TMPV,600.00,1.10,-27.46,69.54,22.98,113945.0,-62042.0,163856.0,-24.30,52.0,X-LC,4.57,2.0,-0.54,1.22,4.75,XY24,NTT,AUTO
73,TCS,4311.58,0.98,-13.26,36.68,18.55,120339.0,-50159.0,328078.0,-25.24,50.0,X-LC,0.54,3.0,-0.42,2.44,11.62,X40,ATH,IT
81,VBL,671.64,-0.32,-5.96,44.27,35.67,131491.0,-18821.0,297021.0,-17.17,37.0,X-LC,1.82,5.0,-0.14,2.21,7.07,X40N,ATH,FMCG
42,INFY,1972.00,-0.38,11.29,20.63,34.24,71836.0,35324.0,348212.0,-13.50,49.0,X-LC,7.13,6.0,0.49,2.59,20.51,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,-0.55,-34.04,101.47,32.89,51676.0,-26280.0,50927.0,-728.69,50.0,L-MC,11.29,259.0,-0.51,0.38,40.25,XR,NTT,BANKS
18,COALINDIA,484.83,-5.65,2.26,16.59,19.23,25526.0,3404.0,153864.0,19.88,48.0,L-LC,12.42,182.0,0.13,1.14,24.02,XY25,ATH,MINING
68,STARHEALTH,761.00,4.77,-8.66,61.23,47.26,157219.0,-24355.0,256768.0,24.55,64.0,H-SC,3.32,174.0,-0.15,1.91,37.57,XY24,NTT,INSURANCE
33,HCLTECH,1908.19,-2.86,8.53,14.11,23.83,37046.0,20627.0,262551.0,13.18,50.0,X-LC,7.29,13.0,0.56,1.95,25.97,X40,ATH,IT
42,INFY,1972.00,-0.38,11.29,20.63,34.24,71836.0,35324.0,348212.0,-13.50,49.0,X-LC,7.13,6.0,0.49,2.59,20.51,X40,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SATIN,274.00,0.15,-14.39,77.35,51.83,182964.0,-39744.0,236540.0,-23.78,61.0,H-SC,3.11,148.0,-0.22,1.76,15.23,XY24,NTT,FINANCE
27,GILLETTE,11206.78,-1.72,2.19,29.58,32.42,76747.0,5559.0,259455.0,-15.97,62.0,X-SC,12.68,87.0,0.07,1.93,17.62,X40,ATH,FMCG
1,ABB,7934.00,-1.52,-1.09,47.17,45.57,122061.0,-2851.0,258768.0,-36.40,64.0,H-MC,6.90,121.0,-0.02,1.93,14.91,AR,NTT,ELECTRICAL
33,HCLTECH,1908.19,-2.86,8.53,14.11,23.83,37046.0,20627.0,262551.0,13.18,50.0,X-LC,7.29,13.0,0.56,1.95,25.97,X40,ATH,IT
68,STARHEALTH,761.00,4.77,-8.66,61.23,47.26,157219.0,-24355.0,256768.0,24.55,64.0,H-SC,3.32,174.0,-0.15,1.91,37.57,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.86
1,25,44.82
2,50,76.80


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.95
MC    30.61
LC    24.43
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.29
X40      24.18
X40N     14.82
AR        8.94
XR        8.70
XY25      8.64
OX40N     7.49
SR        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.95
H-SC    23.06
X-LC    20.90
M-SC    11.00
X-SC    10.14
H-MC     4.81
M-MC     1.47
M-LC     1.24
H-LC     1.15
L-LC     1.14
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.29,-15.10,54.33
FINANCE,13.11,-18.67,65.40
IT,13.02,-22.05,83.03
MISC,6.88,-40.12,97.43
ELECTRICAL,6.01,-16.20,58.59
PAINTS,5.37,-27.74,46.72
INSURANCE,4.89,-3.38,38.75
PHARMA,4.06,-7.90,41.86
BANKS,2.86,-39.22,123.58


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3438264.0,21
AR,1370591.0,10
XR,1321987.0,12
X40,1259278.0,15
X40N,1127816.0,10
OX40N,784190.0,10
XY25,457461.0,6
SR,295306.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3694568.0,24
X-MC,1812170.0,16
M-SC,1579325.0,15
X-LC,1081124.0,12
X-SC,934767.0,9
H-MC,433167.0,3
L-SC,190852.0,2
M-LC,124677.0,1
H-LC,78754.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1262995.0      6
           AR         946192.0      5
M-SC       XY24       899367.0      6
H-SC       XR         855214.0      7
X-MC       X40        613142.0      7
           XY24       496615.0      3
X-LC       X40        466174.0      6
X-MC       X40N       455428.0      4
X-SC       X40N       435803.0      4
M-SC       OX40N      354459.0      5
H-SC       OX40N      334861.0      4
X-SC       XY24       319002.0      3
H-SC       SR         295306.0      2
X-MC       XY25       246985.0      2
X-LC       XY24       241702.0      2
           X40N       236585.0      2
H-MC       XY24       218583.0      1
           AR         214584.0      2
M-SC       XR         194438.0      2
X-SC       X40        179962.0      2
X-LC       XY25       136663.0      2
M-SC       AR         131061.0      2
M-LC       XR         124677.0      1
L-SC       XR          95982.0      1
           OX40N       94870.0      1
H-LC       AR          78754.0      1
L-MC       XR          51676.0      1
M-MC       XY25        48287.0      1
L-LC       XY25        25526.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
